Two previous parts

In [1]:
from bs4 import BeautifulSoup
import requests
import html.parser
import pandas as pd
import numpy as np
!conda install -c conda-forge folium=0.5.0 --yes
import folium
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import csv                  
import webbrowser
import io
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
cols=['PostalCode','Borough','Neighborhood']
session=requests.Session()
r = session.get(url)
soup=BeautifulSoup(r.text, 'html.parser')
table=soup.find('table', class_='wikitable')
output_rows = []
for table_row in table.findAll('tr'):
    columns = table_row.findAll('td')
    output_row = []
    for column in columns:
        output_row.append(column.text)
    output_rows.append(output_row)
output_rows=output_rows[1:]
def stripping(lst):
    return [value.rstrip() for value in lst]
result=list(map(stripping,output_rows))
newresult=list(filter(lambda x: x[1]!='Not assigned',result))
df=pd.DataFrame(newresult, columns=cols)
df.sort_values(by='PostalCode')
coords=pd.read_csv('http://cocl.us/Geospatial_data')
df=df.sort_values(by='PostalCode')
newdf=df.reset_index(drop=True)
newdf['Latitude']=coords['Latitude']
newdf['Longitude']=coords['Longitude']

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    ------------------------------------------------------------
                       

Coords to center a map

In [2]:
from geopy.geocoders import Nominatim
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
# print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

In [ ]:
# Plotting map of these neighborhoods

In [3]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(newdf['Latitude'], newdf['Longitude'], newdf['Borough'], newdf['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Foursquare API credentials

In [4]:
CLIENT_ID = 'VUNTF3ZSXSLZUKZT2E45GMW21D5HJHGK5PVG0Z3MZCTAPWF3' # your Foursquare ID
CLIENT_SECRET = 'GTGDFDX54B2VVS512W5VPQYR5KITNTUNUQ3XUAZIZZZYJVBU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
categories={'Arts & Entertainment': '4d4b7104d754a06370d81259',
            'College & University':'4d4b7105d754a06372d81259',
            'Event':'4d4b7105d754a06373d81259',
            'Food':'4d4b7105d754a06374d81259',
            'Nightlife Spot':'4d4b7105d754a06376d81259',
            'Outdoors & Recreation':'4d4b7105d754a06377d81259',
            'Professional & Other Places':'4d4b7105d754a06375d81259',
            'Residence':'4e67e38e036454776db1fb3a',
            'Travel & Transport':'4d4b7105d754a06379d81259'}



print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: VUNTF3ZSXSLZUKZT2E45GMW21D5HJHGK5PVG0Z3MZCTAPWF3
CLIENT_SECRET:GTGDFDX54B2VVS512W5VPQYR5KITNTUNUQ3XUAZIZZZYJVBU


In [5]:
# FUnction for exploriпg venues in neighborhoods (actually not in, but around it's coordinates)

def explore_NH(neighborhoods, radius=500,LIMIT=100):
    result=[]
    for i,lst in enumerate(zip(neighborhoods['Neighborhood'],neighborhoods['Latitude'],neighborhoods['Longitude'])):
        names=lst[0]
        lat=lst[1]
        long=lst[2]
        to_append=[names,lat,long]
#         for cats in list(categories.items()):
#             categ=cats[0]
#             catId=cats[1]
        url= 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        long, 
        radius, 
        LIMIT)
        request_result=requests.get(url).json()
        result.append(request_result)
#         try:
#             to_append.append(len(request_result['response']['venues'])) 
#         except KeyError:
#             print('----------------------------------------------------------------')
#             print(lat, long)
#             print(requests.get(url).json())
#             print('OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO')
#         result.append(to_append)
    return result
    


Initial results

In [6]:
results = explore_NH(newdf)


Checking results

In [8]:
results[0]['response']['groups'][0]['items'][0]['venue']['categories'][0]['id']

'4bf58dd8d48988d16e941735'

In [10]:
count=0
for i,qwe in enumerate(results):
    if len(qwe['response']['groups'][0]['items'])>0:
        count+=1
print(count)
print(len(results))

100
103


<!-- As you can see there are 100 neighborhoods where are some venues in results.  -->

<!-- Cell below needed to count number of venues and add new column to dataframe -->

In [ ]:
amount=[]
for i in results:
    amount.append(len(i['response']['groups'][0]['items']))

newdf['Venues_count']=amount

print(newdf.head(10))
newdf.tail(10)

I don't want to replicate NY analysis, so here is slightly different analysis

In [91]:
# Firstly I want to get number of venues in different categories for all neighborhoods

# How?

# I need categories' ids from foursquare.com (https://developer.foursquare.com/docs/build-with-foursquare/categories/)
# I need to scrape them (it's not that easy cus of this page on foursquare.com got interesting structure)
# Next step is to transform this information into useful form - dictionary

def check_hierarchy(to_check):
    if to_check.find_all("li")==[]:
        return 0
    else:
        return 1
    
def to_dict(cats):
    dict_={}
    names=[]
    for zero in cats:
        name_0,cat_0 = zero[0]
#         print(name_0)
        names.append([name_0,cat_0])
        dict_[cat_0]=name_0
        for first in zero[1]:
            if type(first[1]) == list:
                dict_[first[0][1]]=zero[0][1]
                for second in first[1]:
                    if type(second[1]) == list:
                        dict_[second[0][1]] = first[0][1]
                        for third in second[1]:
                            dict_[third[1]] = second[0][1]
                    else:
                        dict_[second[1]] =first[0][1]
                    
            else:
                dict_[first[1]] = zero[0][1]
#     for name,cat in names:
#         print(dict_[cat]==name)
#         if name in dict_.values():
#             print(name, ' OK')
#         else:
#             print(name, ' NOT OK')
    return dict_


def get_venue_categories_dict():
    url = 'https://developer.foursquare.com/docs/build-with-foursquare/categories/'
    session = requests.Session()
    r = session.get(url)
    soup = BeautifulSoup(r.text, 'html')
    main_wrapper = soup.find('ul', class_="VenueCategories__Wrapper-sc-1ysxg0y-0 bYmzDC")
    current = main_wrapper.li
    level_0 = []
    while current != None:
        cat_0 = [str(current.h3.string), str(current.p.string)]
        level_0.append(cat_0)
        to_unpack = list(current.div.children)[-1]
        current_1 = to_unpack.li
        level_1 = []
        while current_1 != None:
            cat_1 = [str(current_1.h3.string), str(current_1.p.string)]
            level_1.append(cat_1)
            if check_hierarchy(current_1):
                current_2 = current_1.li
                level_2=[]
                while current_2 != None:
                    cat_2 = [str(current_2.h3.string), str(current_2.p.string)]
                    level_2.append(cat_2)
                    if check_hierarchy(current_2):
                        level_3 = []
                        current_3 = current_2.li
                        while current_3 != None:
                            cat_3 = [str(current_3.h3.string), str(current_3.p.string)]
                            level_3.append(cat_3)
                            current_3 = current_3.next_sibling
                        level_2[-1] = [level_2[-1], level_3]
                    current_2 = current_2.next_sibling
                level_1[-1] = [level_1[-1], level_2]
                                       
            current_1 = current_1.next_sibling
        level_0[-1] = [level_0[-1], level_1]
        current = current.next_sibling
    return to_dict(level_0)




# Then I need to get main category name using venue's category id

def find_main_category(cat, cat_dict):
#     print('CCCCCCCCCAAAAAAAAAAAATTTTTTTTTTTTTT')
#     print(cat)
    if cat_dict[cat] in cat_dict.keys():
        return find_main_category(cat_dict[cat], cat_dict)
    else: return cat_dict[cat]
    
# Now I can get main category name to add columns to dataframe and count number of venues of each category in every NH

def get_categories(data, response, cat_dict):
    for i in range(len(data)):
        ids = []
        for venue in results[i]['response']['groups'][0]['items']:
            ids.append(venue['venue']['categories'][0]['id'])
        dict0={}
        for id_ in ids:
            ID = find_main_category(id_, cat_dict)
            if ID in dict0.keys():
                dict0[ID] += 1
            else:
                dict0[ID] = 1
        for name, val in dict0.items():
            data.loc[i,name] = val
    return data

In [ ]:
newdf.drop(labels='ids', axis=1, inplace=True)
get_categories(newdf, results, get_venue_categories_dict())

In [98]:
newdf.fillna(0, inplace=True)
newdf

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Venues_count,Food,Outdoors & Recreation,Shop & Service,Nightlife Spot,Travel & Transport,Professional & Other Places,Arts & Entertainment,College & University,Residence
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,3,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,8,3.0,0.0,2.0,0.0,2.0,1.0,0.0,0.0,0.0
3,M1G,Scarborough,Woburn,43.770992,-79.216917,3,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,9,5.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029,7,2.0,0.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577,10,3.0,2.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476,3,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,4,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


Now I will go right to clustering because frequency analysis here is pretty irrelevant

In [102]:
from sklearn.cluster import KMeans

kclusters = 6

kmeans = KMeans(n_clusters=kclusters, random_state=13).fit(newdf.iloc[:, -9:])
# check cluster labels generated for each row in the dataframe
kmeans.labels_[20:40]

array([0, 0, 1, 0, 0, 0, 0, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3],
      dtype=int32)

In [107]:
# add clustering labels

merged=pd.DataFrame(index=newdf.index)
merged['Latitude']=newdf['Latitude']
merged['Longitude']=newdf['Longitude']
merged['Cluster Labels']=kmeans.labels_
newdf.insert(newdf.shape[1], 'Cluster Labels', kmeans.labels_)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged.head() # check the last columns!



,Latitude,Longitude,Cluster Labels
0,43.806686,-79.194353,0
1,43.784535,-79.160497,0
2,43.763573,-79.188711,0
3,43.770992,-79.216917,0
4,43.773136,-79.239476,0


In [104]:
# create map

import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['Latitude'], merged['Longitude'], merged.index, merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [108]:
first=newdf[newdf['Cluster Labels']==0]
second=newdf[newdf['Cluster Labels']==1]
third=newdf[newdf['Cluster Labels']==2]
fourth=newdf[newdf['Cluster Labels']==3]
fifth=newdf[newdf['Cluster Labels']==4]

In [109]:
lst=[first,second,third,fourth,fifth]
for i in lst:
    i.drop('Latitude', axis=1, inplace=True)
    i.drop('Longitude', axis=1, inplace=True)
    i.drop('Cluster Labels', axis=1, inplace=True)
    print(i.head())


/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


  PostalCode      Borough                            Neighborhood  \
0        M1B  Scarborough                          Malvern, Rouge   
1        M1C  Scarborough  Rouge Hill, Port Union, Highland Creek   
2        M1E  Scarborough       Guildwood, Morningside, West Hill   
3        M1G  Scarborough                                  Woburn   
4        M1H  Scarborough                               Cedarbrae   

   Venues_count  Food  Outdoors & Recreation  Shop & Service  Nightlife Spot  \
0             1   1.0                    0.0             0.0             0.0   
1             3   0.0                    1.0             1.0             1.0   
2             8   3.0                    0.0             2.0             0.0   
3             3   3.0                    0.0             0.0             0.0   
4             9   5.0                    1.0             2.0             1.0   

   Travel & Transport  Professional & Other Places  Arts & Entertainment  \
0                 0.0       

In [ ]:
qwe2=qwe.find("li", recursive=False)

for i in qwe2.find_all("ul"):
#     print('0000000000000000000000000000000000000')
#     print(i)
#     print('0000000000000000000000000000000000000')
    try:
        print(i.h3)
        print(i.p)
    except: continue
#     print('0000000000000000000000000000000000000')
#     print(i.h3)
#     print(i.p)
#     try:
#         print(i.next_sibling.h3)
#     except: print('NOOOO')
#     for j in i.descendants:
#         print('--------------------')
#         print(j.find_all("ul"))
#     print('0000000000000000000000000000000000000')

In [ ]:
for i in qwe2.children:
    print(i.h3)

In [ ]:
len(list(qwe2.children))

In [ ]:
print(qwe2.prettify())

In [ ]:
qwe2.div.children

In [ ]:
for i in qwe2.find_all("li", recursive=True):
    print('0000000000000000000000000000000000000')
    print(i.h3)
    if len(list(i.children))==1:
        print('11111111111111111111')
    else:
        for j in i.children:
            print('222222222222222222222')
            print(j.h3)
    print('0000000000000000000000000000000000000')

In [ ]:
qqq=list(qwe2.find_all("li", recursive=True))[19]

In [ ]:
qqq

In [ ]:
list(qqq.div.children)

In [ ]:
qqq.h3

In [ ]:
for i in qqq.div.children:
    print(i.find_all("h3"))
    

In [ ]:
sss=list(qqq.div.children)[-1]

In [ ]:
ddd=list(sss.children)

In [ ]:
for i in ddd:
    print(i.h3)
    print(i.p)
def    print(list(i.children))

In [15]:
def check_hierarchy(to_check):
    if to_check.find_all("li")==[]:
        return 0
    else:
        return 1
    

        
def get_venue_categories_dict():
    url = 'https://developer.foursquare.com/docs/build-with-foursquare/categories/'
    session = requests.Session()
    r = session.get(url)
    soup = BeautifulSoup(r.text, 'html')
    main_wrapper = soup.find('ul', class_="VenueCategories__Wrapper-sc-1ysxg0y-0 bYmzDC")
    current = main_wrapper.li
    level_0 = []
    while current != None:
        cat_0 = [str(current.h3.string), str(current.p.string)]
        level_0.append(cat_0)
        to_unpack = list(current.div.children)[-1]
        current_1 = to_unpack.li
        level_1 = []
        while current_1 != None:
            cat_1 = [str(current_1.h3.string), str(current_1.p.string)]
            level_1.append(cat_1)
            if check_hierarchy(current_1):
                current_2 = current_1.li
                level_2=[]
                while current_2 != None:
                    cat_2 = [str(current_2.h3.string), str(current_2.p.string)]
                    level_2.append(cat_2)
                    if check_hierarchy(current_2):
                        level_3 = []
                        current_3 = current_2.li
                        while current_3 != None:
                            cat_3 = [str(current_3.h3.string), str(current_3.p.string)]
                            level_3.append(cat_3)
                            current_3 = current_3.next_sibling
                        level_2[-1] = [level_2[-1], level_3]
                    current_2 = current_2.next_sibling
                level_1[-1] = [level_1[-1], level_2]
                                       
            current_1 = current_1.next_sibling
        level_0[-1] = [level_0[-1], level_1]
        current = current.next_sibling
    return to_dict(level_0)


In [16]:
dididi=get_venue_categories_dict()

Arts & Entertainment
College & University
Event
Food
Nightlife Spot
Outdoors & Recreation
Professional & Other Places
Residence
Shop & Service
Travel & Transport
True
Arts & Entertainment  OK
True
College & University  OK
True
Event  OK
True
Food  OK
True
Nightlife Spot  OK
True
Outdoors & Recreation  OK
True
Professional & Other Places  OK
True
Residence  OK
True
Shop & Service  OK
True
Travel & Transport  OK


In [9]:
lst=[]
for zero in dididi:
    print(zero[0])
    lst.append([zero[0][1],zero[0][0]])
    for first in zero[1]:
        if type(first[1]) == list:
            print("\t",first[0])
            lst.append([first[0][1],zero[0][1]])
            for second in first[1]:
                if type(second[1]) == list:
                    print("\t\t", second[0])
                    lst.append([second[0][1],first[0][1]])
                    for third in second[1]:
                        print("\t\t\t", third)
                        lst.append([third[1],first[0][1]])
                else:
                    print("\t\t", second)
                    lst.append([second[1], first[0][1]])
        else:
            print("\t", first)
            lst.append([first[1],zero[0][1]])
            
for i in lst:
    print(i)
            

['Arts & Entertainment', '4d4b7104d754a06370d81259']
	 ['Amphitheater', '56aa371be4b08b9a8d5734db']
	 ['Aquarium', '4fceea171983d5d06c3e9823']
	 ['Arcade', '4bf58dd8d48988d1e1931735']
	 ['Art Gallery', '4bf58dd8d48988d1e2931735']
	 ['Bowling Alley', '4bf58dd8d48988d1e4931735']
	 ['Casino', '4bf58dd8d48988d17c941735']
	 ['Circus', '52e81612bcbc57f1066b79e7']
	 ['Comedy Club', '4bf58dd8d48988d18e941735']
	 ['Concert Hall', '5032792091d4c4b30a586d5c']
	 ['Country Dance Club', '52e81612bcbc57f1066b79ef']
	 ['Disc Golf', '52e81612bcbc57f1066b79e8']
	 ['Exhibit', '56aa371be4b08b9a8d573532']
	 ['General Entertainment', '4bf58dd8d48988d1f1931735']
	 ['Go Kart Track', '52e81612bcbc57f1066b79ea']
	 ['Historic Site', '4deefb944765f83613cdba6e']
	 ['Karaoke Box', '5744ccdfe4b0c0459246b4bb']
	 ['Laser Tag', '52e81612bcbc57f1066b79e6']
	 ['Memorial Site', '5642206c498e4bfca532186c']
	 ['Mini Golf', '52e81612bcbc57f1066b79eb']
	 ['Movie Theater', '4bf58dd8d48988d17f941735']
		 ['Drive-in Theater', '5

	 ['Photography Studio', '554a5e17498efabeda6cc559']
	 ['Piercing Parlor', '52f2ab2ebcbc57f1066b8b20']
	 ['Pop-Up Shop', '52f2ab2ebcbc57f1066b8b3d']
	 ['Print Shop', '52f2ab2ebcbc57f1066b8b28']
	 ['Public Bathroom', '5744ccdfe4b0c0459246b4c4']
	 ['Real Estate Office', '5032885091d4c4b30a586d66']
	 ['Record Shop', '4bf58dd8d48988d10d951735']
	 ['Recording Studio', '52f2ab2ebcbc57f1066b8b37']
	 ['Recycling Facility', '4f4531084b9074f6e4fb0101']
	 ['Rental Service', '56aa371be4b08b9a8d573552']
	 ['Salon / Barbershop', '4bf58dd8d48988d110951735']
	 ['Sauna / Steam Room', '58daa1558bbb0b01f18ec1ae']
	 ['Shipping Store', '52f2ab2ebcbc57f1066b8b1f']
	 ['Shoe Repair', '52f2ab2ebcbc57f1066b8b39']
	 ['Shopping Mall', '4bf58dd8d48988d1fd941735']
	 ['Shopping Plaza', '5744ccdfe4b0c0459246b4dc']
	 ['Skate Shop', '5bae9231bedf3950379f89d2']
	 ['Ski Shop', '56aa371be4b08b9a8d573566']
	 ['Smoke Shop', '4bf58dd8d48988d123951735']
	 ['Smoothie Shop', '52f2ab2ebcbc57f1066b8b41']
	 ['Souvenir Shop', '52f2

In [17]:
dididi


{'4d4b7104d754a06370d81259': 'Arts & Entertainment',
 '56aa371be4b08b9a8d5734db': '4d4b7104d754a06370d81259',
 '4fceea171983d5d06c3e9823': '4d4b7104d754a06370d81259',
 '4bf58dd8d48988d1e1931735': '4d4b7104d754a06370d81259',
 '4bf58dd8d48988d1e2931735': '4d4b7104d754a06370d81259',
 '4bf58dd8d48988d1e4931735': '4d4b7104d754a06370d81259',
 '4bf58dd8d48988d17c941735': '4d4b7104d754a06370d81259',
 '52e81612bcbc57f1066b79e7': '4d4b7104d754a06370d81259',
 '4bf58dd8d48988d18e941735': '4d4b7104d754a06370d81259',
 '5032792091d4c4b30a586d5c': '4d4b7104d754a06370d81259',
 '52e81612bcbc57f1066b79ef': '4d4b7104d754a06370d81259',
 '52e81612bcbc57f1066b79e8': '4d4b7104d754a06370d81259',
 '56aa371be4b08b9a8d573532': '4d4b7104d754a06370d81259',
 '4bf58dd8d48988d1f1931735': '4d4b7104d754a06370d81259',
 '52e81612bcbc57f1066b79ea': '4d4b7104d754a06370d81259',
 '4deefb944765f83613cdba6e': '4d4b7104d754a06370d81259',
 '5744ccdfe4b0c0459246b4bb': '4d4b7104d754a06370d81259',
 '52e81612bcbc57f1066b79e6': '4d4b7

In [10]:
cash=[]
for i in lst:
    print(i[0])
    print(i[0] in cash)
    cash.append(i[0])
     

4d4b7104d754a06370d81259
False
56aa371be4b08b9a8d5734db
False
4fceea171983d5d06c3e9823
False
4bf58dd8d48988d1e1931735
False
4bf58dd8d48988d1e2931735
False
4bf58dd8d48988d1e4931735
False
4bf58dd8d48988d17c941735
False
52e81612bcbc57f1066b79e7
False
4bf58dd8d48988d18e941735
False
5032792091d4c4b30a586d5c
False
52e81612bcbc57f1066b79ef
False
52e81612bcbc57f1066b79e8
False
56aa371be4b08b9a8d573532
False
4bf58dd8d48988d1f1931735
False
52e81612bcbc57f1066b79ea
False
4deefb944765f83613cdba6e
False
5744ccdfe4b0c0459246b4bb
False
52e81612bcbc57f1066b79e6
False
5642206c498e4bfca532186c
False
52e81612bcbc57f1066b79eb
False
4bf58dd8d48988d17f941735
False
56aa371be4b08b9a8d5734de
False
4bf58dd8d48988d17e941735
False
4bf58dd8d48988d180941735
False
4bf58dd8d48988d181941735
False
4bf58dd8d48988d18f941735
False
559acbe0498e472f1a53fa23
False
4bf58dd8d48988d190941735
False
4bf58dd8d48988d192941735
False
4bf58dd8d48988d191941735
False
4bf58dd8d48988d1e5931735
False
4bf58dd8d48988d1e7931735
False
4bf58dd8

In [12]:
def to_dict(cats):
    dict_={}
    names=[]
    for zero in cats:
        name_0,cat_0 = zero[0]
        print(name_0)
        names.append([name_0,cat_0])
        dict_[cat_0]=name_0
        for first in zero[1]:
            if type(first[1]) == list:
                dict_[first[0][1]]=zero[0][1]
                for second in first[1]:
                    if type(second[1]) == list:
                        dict_[second[0][1]] = first[0][1]
                        for third in second[1]:
                            dict_[third[1]] = second[0][1]
                    else:
                        dict_[second[1]] =first[0][1]
                    
            else:
                dict_[first[1]] = zero[0][1]
    for name,cat in names:
        print(dict_[cat]==name)
        if name in dict_.values():
            print(name, ' OK')
        else:
            print(name, ' NOT OK')
    return dict_



gigigi = dididi[2:4]


to_dict(dididi)

# print(len(gigigi))
# print(len(gigigi[1]))
# print(len(gigigi[1][1]))
# print(gigigi[1][1][3])
# print(len(gigigi[1][1][3]))
# print(gigigi[1][1][3][0])
# print(len(gigigi[1][1][3][1]))
# print(gigigi[1][1][3][1][2])
# print(len(gigigi[1][1][3][1][2]))
# print(gigigi[1][1][3][1][2][0])
# print(len(gigigi[1][1][3][1][2][1]))
# print(gigigi[1][1][3][1][2][1][0])

# print(len(dd.keys()))
# for i in dd.keys():
#     print(find_main_category(i,dd))
# asasas = to_dict(dididi)

Arts & Entertainment
College & University
Event
Food
Nightlife Spot
Outdoors & Recreation
Professional & Other Places
Residence
Shop & Service
Travel & Transport
True
Arts & Entertainment  OK
True
College & University  OK
True
Event  OK
True
Food  OK
True
Nightlife Spot  OK
True
Outdoors & Recreation  OK
True
Professional & Other Places  OK
True
Residence  OK
True
Shop & Service  OK
True
Travel & Transport  OK


{'4d4b7104d754a06370d81259': 'Arts & Entertainment',
 '56aa371be4b08b9a8d5734db': '4d4b7104d754a06370d81259',
 '4fceea171983d5d06c3e9823': '4d4b7104d754a06370d81259',
 '4bf58dd8d48988d1e1931735': '4d4b7104d754a06370d81259',
 '4bf58dd8d48988d1e2931735': '4d4b7104d754a06370d81259',
 '4bf58dd8d48988d1e4931735': '4d4b7104d754a06370d81259',
 '4bf58dd8d48988d17c941735': '4d4b7104d754a06370d81259',
 '52e81612bcbc57f1066b79e7': '4d4b7104d754a06370d81259',
 '4bf58dd8d48988d18e941735': '4d4b7104d754a06370d81259',
 '5032792091d4c4b30a586d5c': '4d4b7104d754a06370d81259',
 '52e81612bcbc57f1066b79ef': '4d4b7104d754a06370d81259',
 '52e81612bcbc57f1066b79e8': '4d4b7104d754a06370d81259',
 '56aa371be4b08b9a8d573532': '4d4b7104d754a06370d81259',
 '4bf58dd8d48988d1f1931735': '4d4b7104d754a06370d81259',
 '52e81612bcbc57f1066b79ea': '4d4b7104d754a06370d81259',
 '4deefb944765f83613cdba6e': '4d4b7104d754a06370d81259',
 '5744ccdfe4b0c0459246b4bb': '4d4b7104d754a06370d81259',
 '52e81612bcbc57f1066b79e6': '4d4b7

In [18]:
def find_main_category(cat, cat_dict):
    print('CCCCCCCCCAAAAAAAAAAAATTTTTTTTTTTTTT')
    print(cat)
    if cat_dict[cat] in cat_dict.keys():
        return find_main_category(cat_dict[cat], cat_dict)
    else: return cat_dict[cat]

In [19]:
dididi = get_venue_categories_dict()

Arts & Entertainment
College & University
Event
Food
Nightlife Spot
Outdoors & Recreation
Professional & Other Places
Residence
Shop & Service
Travel & Transport
True
Arts & Entertainment  OK
True
College & University  OK
True
Event  OK
True
Food  OK
True
Nightlife Spot  OK
True
Outdoors & Recreation  OK
True
Professional & Other Places  OK
True
Residence  OK
True
Shop & Service  OK
True
Travel & Transport  OK


In [24]:
find_main_category('56aa371be4b08b9a8d573556', dididi)

CCCCCCCCCAAAAAAAAAAAATTTTTTTTTTTTTT
56aa371be4b08b9a8d573556
CCCCCCCCCAAAAAAAAAAAATTTTTTTTTTTTTT
4bf58dd8d48988d184941735
CCCCCCCCCAAAAAAAAAAAATTTTTTTTTTTTTT
4d4b7104d754a06370d81259


'Arts & Entertainment'

In [23]:
# dididi['6aa371be4b08b9a8d573556']
dididi

{'4d4b7104d754a06370d81259': 'Arts & Entertainment',
 '56aa371be4b08b9a8d5734db': '4d4b7104d754a06370d81259',
 '4fceea171983d5d06c3e9823': '4d4b7104d754a06370d81259',
 '4bf58dd8d48988d1e1931735': '4d4b7104d754a06370d81259',
 '4bf58dd8d48988d1e2931735': '4d4b7104d754a06370d81259',
 '4bf58dd8d48988d1e4931735': '4d4b7104d754a06370d81259',
 '4bf58dd8d48988d17c941735': '4d4b7104d754a06370d81259',
 '52e81612bcbc57f1066b79e7': '4d4b7104d754a06370d81259',
 '4bf58dd8d48988d18e941735': '4d4b7104d754a06370d81259',
 '5032792091d4c4b30a586d5c': '4d4b7104d754a06370d81259',
 '52e81612bcbc57f1066b79ef': '4d4b7104d754a06370d81259',
 '52e81612bcbc57f1066b79e8': '4d4b7104d754a06370d81259',
 '56aa371be4b08b9a8d573532': '4d4b7104d754a06370d81259',
 '4bf58dd8d48988d1f1931735': '4d4b7104d754a06370d81259',
 '52e81612bcbc57f1066b79ea': '4d4b7104d754a06370d81259',
 '4deefb944765f83613cdba6e': '4d4b7104d754a06370d81259',
 '5744ccdfe4b0c0459246b4bb': '4d4b7104d754a06370d81259',
 '52e81612bcbc57f1066b79e6': '4d4b7

In [69]:
dididi['52e928d0bcbc57f1066b7e96']

'4d4b7105d754a06374d81259'

In [14]:
find_main_category('52af3a5e3cf9994f4e043bea', dididi)

CCCCCCCCCAAAAAAAAAAAATTTTTTTTTTTTTT
52af3a5e3cf9994f4e043bea


TypeError: list indices must be integers or slices, not str

In [ ]:
def to_array(venues_list):
    classes = np.array([i for i in venues_list])

rty=np.array(rty)
rty.flatten()

In [ ]:
# for i in current.div.children:
#     print(i)

list(current.div.children)[-1].li.find_all("li")

In [ ]:
print(list(current.div.children)[-1].prettify())

In [ ]:
movies=list(current.div.children)[-1].li.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling

In [ ]:
len(list(movies.children))

In [ ]:
list(movies.children)[0].li.next_sibling.next_sibling.next_sibling==None

In [ ]:
len(list(list(current.div.children)[-1].li.next_sibling.children))

In [ ]:
len(list(current.div.children))

In [ ]:
list(current.div.children)[0]

In [ ]:
list(current.div.children)[1]

In [ ]:
list(current.div.children)[-1].find_all("li")

In [ ]:
len(list(list(current.div.children)[-1].li.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.children))

In [ ]:
current.div.li

In [ ]:
lst=[1,2,3]


In [ ]:
lst[-1].append([1,2,3])
lst